# Simple VGG implementation
## Loading and augmenting the data

In [1]:
# Trying to use less decimals for testing:
# it works nice on my 2GB laptop!
import keras.backend as K
K.set_floatx('float16')
#K.set_epsilon(1e-4)

Using TensorFlow backend.


In [10]:
from keras.preprocessing.image import ImageDataGenerator

experiment = '0.2.1'

train_path = '/data/resized_224/train'
validation_path = '/data/resized_224/validation'
epochs = 10
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary') 

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_directory(
        validation_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary') 


Found 1600 images belonging to 2 classes.
Found 384 images belonging to 2 classes.


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))




In [11]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss = 'binary_crossentropy',
              optimizer = sgd,
              metrics = ['accuracy'])

In [12]:
from keras import callbacks
tbCallBack = callbacks.TensorBoard(log_dir='/code/logs/{}'.format(experiment))


In [13]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=800 // batch_size,
        callbacks=[tbCallBack])

Epoch 1/10
125/125 [==============================] - 179s 1s/step - loss: 0.7069 - acc: 0.4885 - val_loss: 0.7160 - val_acc: 0.5000
Epoch 2/10
125/125 [==============================] - 175s 1s/step - loss: 0.7023 - acc: 0.4800 - val_loss: 0.6951 - val_acc: 0.5000
Epoch 3/10
125/125 [==============================] - 175s 1s/step - loss: 0.7001 - acc: 0.5000 - val_loss: 0.7037 - val_acc: 0.5038
Epoch 4/10
125/125 [==============================] - 174s 1s/step - loss: 0.7034 - acc: 0.4785 - val_loss: 0.6947 - val_acc: 0.4938
Epoch 5/10
 95/125 [=====================>........] - ETA: 37s - loss: 0.6989 - acc: 0.4717

KeyboardInterrupt: 